In [258]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import classification_report

In [259]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import nltk
import string
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
nltk.download('stopwords')
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [260]:
df_train = pd.read_csv('train_data.csv')
df_test = pd.read_csv('test_data.csv')

In [261]:
df = pd.concat([df_train, df_test])

In [262]:
df.reset_index(inplace=True)

In [263]:
df

,index,text,TIER_1_WHICH_HALF,TIER_2_LETTER_RANGE,TIER_3_ACTUAL_LETTER
0,0,What country was Mikhail Gorbachev the leader ...,LAST 12 LETTERS,BETWEEN T AND Z,t
1,1,What company uses the unfamiliar faces of its ...,FIRST 14 LETTERS,BETWEEN A AND G,d
2,2,Name the Four Horsemen of the Apocalypse .,FIRST 14 LETTERS,BETWEEN H AND N,j
3,3,How much is a Canadian 1967 twenty dollar gold...,FIRST 14 LETTERS,BETWEEN H AND N,h
4,4,What city has the two steepest streets in the ...,LAST 12 LETTERS,BETWEEN T AND Z,v
...,...,...,...,...,...
70,11,What is hyperopia ?,LAST 12 LETTERS,BETWEEN T AND Z,v
71,12,What 1920s cowboy star rode Tony the Wonder Ho...,LAST 12 LETTERS,BETWEEN T AND Z,v
72,13,"What is a fear of odors , body , ?",LAST 12 LETTERS,BETWEEN T AND Z,x
73,14,What layer of the atmosphere contains most of ...,LAST 12 LETTERS,BETWEEN T AND Z,z


In [264]:
df.drop(columns=['index'],inplace=True)

In [265]:
# Number of stop words: 
df['stopwords'] = df['text'].apply(lambda x: len([x for x in x.split() if x in stop]))
df[['text','stopwords']].head()

# The number of punctuation:
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return count

df['punctuation'] = df['text'].apply(lambda x: count_punct(x))
df[['text','punctuation']].head()

# Number of numerical characters:
df['numerics'] = df['text'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
df[['text','numerics']].head()

#Number of Uppercase words:
df['upper'] = df['text'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
df[['text','upper']].head()

# Make all text lower case:
df['text'] = df['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['text'].head()

# Removing Punctuation:
df['text'] = df['text'].str.replace('[^\w\s]','')
df['text'].head()

# Removal of Stop Words
df['text'] = df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df['text'].sample(10)

# Number of Words
df['word_count'] = df['text'].apply(lambda x: len(str(x).split(" ")))
df[['text','word_count']].head()

# Number of characters
df['char_count'] = df['text'].str.len() ## this also includes spaces
df[['text','char_count']].head()

# Average Word Length
def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/(len(words)+0.000001))
df['avg_word'] = df['text'].apply(lambda x: avg_word(x)).round(1)
df[['text','avg_word']].head()

<ipython-input-265-19b070ca13a6>:26: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace('[^\w\s]','')


,text,avg_word
0,country mikhail gorbachev leader,7.2
1,company uses unfamiliar faces celebrity spokes...,7.1
2,name four horsemen apocalypse,6.5
3,much canadian 1967 twenty dollar gold coin worth,5.1
4,city two steepest streets us,4.8


In [266]:
# df = df.drop(columns=['Unnamed: 0'])

In [267]:
df.head()

,text,TIER_1_WHICH_HALF,TIER_2_LETTER_RANGE,TIER_3_ACTUAL_LETTER,stopwords,punctuation,numerics,upper,word_count,char_count,avg_word
0,country mikhail gorbachev leader,LAST 12 LETTERS,BETWEEN T AND Z,t,3,1,0,0,4,32,7.2
1,company uses unfamiliar faces celebrity spokes...,FIRST 14 LETTERS,BETWEEN A AND G,d,7,1,0,0,9,72,7.1
2,name four horsemen apocalypse,FIRST 14 LETTERS,BETWEEN H AND N,j,3,1,0,0,4,29,6.5
3,much canadian 1967 twenty dollar gold coin worth,FIRST 14 LETTERS,BETWEEN H AND N,h,2,1,1,0,8,48,5.1
4,city two steepest streets us,LAST 12 LETTERS,BETWEEN T AND Z,v,4,3,0,1,5,28,4.8


In [268]:
vectorizer = TfidfVectorizer(analyzer='char')
count_vect_df = pd.DataFrame(vectorizer.fit_transform(df['text']).todense(), columns=vectorizer.get_feature_names_out())
df = pd.concat([df, count_vect_df], axis=1)

In [269]:
df.head()

,text,TIER_1_WHICH_HALF,TIER_2_LETTER_RANGE,TIER_3_ACTUAL_LETTER,stopwords,punctuation,numerics,upper,word_count,char_count,...,q,r,s,t,u,v,w,x,y,z
0,country mikhail gorbachev leader,LAST 12 LETTERS,BETWEEN T AND Z,t,3,1,0,0,4,32,...,0.0,0.336223,0.000000,0.100729,0.160932,0.243991,0.000000,0.0,0.158379,0.0
1,company uses unfamiliar faces celebrity spokes...,FIRST 14 LETTERS,BETWEEN A AND G,d,7,1,0,0,9,72,...,0.0,0.196076,0.286313,0.132170,0.281554,0.000000,0.000000,0.0,0.138543,0.0
2,name four horsemen apocalypse,FIRST 14 LETTERS,BETWEEN H AND N,j,3,1,0,0,4,29,...,0.0,0.230939,0.224815,0.000000,0.165808,0.000000,0.000000,0.0,0.163177,0.0
3,much canadian 1967 twenty dollar gold coin worth,FIRST 14 LETTERS,BETWEEN H AND N,h,2,1,1,0,8,48,...,0.0,0.143441,0.000000,0.193379,0.102986,0.000000,0.249745,0.0,0.101352,0.0
4,city two steepest streets us,LAST 12 LETTERS,BETWEEN T AND Z,v,4,3,0,1,5,28,...,0.0,0.100690,0.490097,0.542980,0.144585,0.000000,0.175311,0.0,0.142291,0.0


In [270]:
# train_ratio = 0.8
# total_rows = df.shape[0]
# df_train = df.sample(n=int(total_rows*train_ratio), random_state=12)
# df_test=df.drop(df_train.index)
df_train = df.head(59)
df_test = df.tail(16)

In [271]:
# Predicting TIER_1_WHICH_HALF

In [272]:
train_x_1 = df_train.drop(columns=['text','TIER_1_WHICH_HALF','TIER_2_LETTER_RANGE','TIER_3_ACTUAL_LETTER'])
test_x_1 = df_test.drop(columns=['text','TIER_1_WHICH_HALF','TIER_2_LETTER_RANGE','TIER_3_ACTUAL_LETTER'])
train_y_1 = df_train['TIER_1_WHICH_HALF']
test_y_1 = df_test['TIER_1_WHICH_HALF']

In [273]:
train_y_1.value_counts()

FIRST 14 LETTERS    32
LAST 12 LETTERS     27
Name: TIER_1_WHICH_HALF, dtype: int64

In [274]:
model_1 = SVC()
# model_1 = MultinomialNB()
# model_1 = GaussianNB()
# model_1 = DecisionTreeClassifier()
# model_1 = LogisticRegression()
model_1.fit(train_x_1,train_y_1)

SVC()

In [275]:
pred_y_1 = model_1.predict(test_x_1)
print(classification_report(test_y_1, pred_y_1))

                  precision    recall  f1-score   support

FIRST 14 LETTERS       0.67      0.50      0.57         8
 LAST 12 LETTERS       0.60      0.75      0.67         8

        accuracy                           0.62        16
       macro avg       0.63      0.62      0.62        16
    weighted avg       0.63      0.62      0.62        16



In [276]:
test_y_1

59    FIRST 14 LETTERS
60    FIRST 14 LETTERS
61    FIRST 14 LETTERS
62    FIRST 14 LETTERS
63    FIRST 14 LETTERS
64    FIRST 14 LETTERS
65    FIRST 14 LETTERS
66    FIRST 14 LETTERS
67     LAST 12 LETTERS
68     LAST 12 LETTERS
69     LAST 12 LETTERS
70     LAST 12 LETTERS
71     LAST 12 LETTERS
72     LAST 12 LETTERS
73     LAST 12 LETTERS
74     LAST 12 LETTERS
Name: TIER_1_WHICH_HALF, dtype: object

In [277]:
pred_y_1

array(['FIRST 14 LETTERS', 'LAST 12 LETTERS', 'LAST 12 LETTERS',
       'LAST 12 LETTERS', 'LAST 12 LETTERS', 'FIRST 14 LETTERS',
       'FIRST 14 LETTERS', 'FIRST 14 LETTERS', 'LAST 12 LETTERS',
       'LAST 12 LETTERS', 'LAST 12 LETTERS', 'LAST 12 LETTERS',
       'FIRST 14 LETTERS', 'LAST 12 LETTERS', 'FIRST 14 LETTERS',
       'LAST 12 LETTERS'], dtype=object)

In [278]:
df_test['TIER_1_WHICH_HALF_PRED'] = pred_y_1

<ipython-input-278-347bbb580985>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['TIER_1_WHICH_HALF_PRED'] = pred_y_1


In [279]:
#Predicting TIER_2_LETTER_RANGE given TIER_1_WHICH_HALF = FIRST 14 LETTERS

In [280]:
# taking train data having TIER_1_WHICH_HALF = FIRST 14 LETTERS
train_x_2_a = df_train[df_train['TIER_1_WHICH_HALF'][train_x_1.index] == 'FIRST 14 LETTERS'].drop(columns=['text','TIER_1_WHICH_HALF','TIER_2_LETTER_RANGE','TIER_3_ACTUAL_LETTER'])
# taking test data where TIER_1_WHICH_HALF was predicted as FIRST 14 LETTERS
test_x_2_a = df_test[df_test['TIER_1_WHICH_HALF'][test_x_1.index] == 'FIRST 14 LETTERS'].drop(columns=['text','TIER_1_WHICH_HALF','TIER_2_LETTER_RANGE','TIER_3_ACTUAL_LETTER','TIER_1_WHICH_HALF_PRED'])

train_y_2_a = df_train[df_train['TIER_1_WHICH_HALF'][train_x_1.index] == 'FIRST 14 LETTERS']['TIER_2_LETTER_RANGE']
test_y_2_a = df_test[df_test['TIER_1_WHICH_HALF'][test_x_1.index] == 'FIRST 14 LETTERS']['TIER_2_LETTER_RANGE']

In [281]:
train_y_2_a.value_counts()

BETWEEN A AND G    22
BETWEEN H AND N    10
Name: TIER_2_LETTER_RANGE, dtype: int64

In [282]:
test_y_2_a.value_counts()

BETWEEN A AND G    5
BETWEEN H AND N    3
Name: TIER_2_LETTER_RANGE, dtype: int64

In [283]:
#from imblearn.datasets import make_imbalance
#train_x_2_a, train_y_2_a = make_imbalance(train_x_2_a, train_y_2_a, sampling_strategy={'BETWEEN A AND G': 10, 'BETWEEN H AND N': 10},random_state=14)


In [288]:
model_first14 = SVC()
# model_first14 = MultinomialNB()
# model_first14 = GaussianNB()
# model_first14 = DecisionTreeClassifier()
# model_first14 = LogisticRegression()
model_first14.fit(train_x_2_a,train_y_2_a)

SVC()

In [289]:
pred_y_2_a = model_first14.predict(test_x_2_a)
print(classification_report(test_y_2_a, pred_y_2_a))

                 precision    recall  f1-score   support

BETWEEN A AND G       0.62      1.00      0.77         5
BETWEEN H AND N       0.00      0.00      0.00         3

       accuracy                           0.62         8
      macro avg       0.31      0.50      0.38         8
   weighted avg       0.39      0.62      0.48         8



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [290]:
pred_y_2_a

array(['BETWEEN A AND G', 'BETWEEN A AND G', 'BETWEEN A AND G',
       'BETWEEN A AND G', 'BETWEEN A AND G', 'BETWEEN A AND G',
       'BETWEEN A AND G', 'BETWEEN A AND G'], dtype=object)

In [291]:
#Predicting TIER_2_LETTER_RANGE given TIER_1_WHICH_HALF = LAST 12 LETTERS

In [292]:
# taking train data having TIER_1_WHICH_HALF = LAST 12 LETTERS
train_x_2_b = df_train[df_train['TIER_1_WHICH_HALF'][train_x_1.index] == 'LAST 12 LETTERS'].drop(columns=['text','TIER_1_WHICH_HALF','TIER_2_LETTER_RANGE','TIER_3_ACTUAL_LETTER'])
# taking test data where TIER_1_WHICH_HALF was predicted as LAST 12 LETTERS
test_x_2_b = df_test[df_test['TIER_1_WHICH_HALF'][test_x_1.index] == 'LAST 12 LETTERS'].drop(columns=['text','TIER_1_WHICH_HALF','TIER_2_LETTER_RANGE','TIER_3_ACTUAL_LETTER','TIER_1_WHICH_HALF_PRED'])

# taking train Y having TIER_1_WHICH_HALF = LAST 12 LETTERS
train_y_2_b = df_train[df_train['TIER_1_WHICH_HALF'][train_x_1.index] == 'LAST 12 LETTERS']['TIER_2_LETTER_RANGE']
# taking test Y where TIER_1_WHICH_HALF was predicted as LAST 12 LETTERS
test_y_2_b = df_test[df_test['TIER_1_WHICH_HALF'][test_x_1.index] == 'LAST 12 LETTERS']['TIER_2_LETTER_RANGE']

In [293]:
# model_last12 = SVC()
model_last12 = MultinomialNB()
# model_last12 = GaussianNB()
# model_last12 = DecisionTreeClassifier()
# model_last12 = LogisticRegression()
model_last12.fit(train_x_2_b,train_y_2_b)

MultinomialNB()

In [294]:
pred_y_2_b = model_last12.predict(test_x_2_b)
print(classification_report(test_y_2_b, pred_y_2_b))

                 precision    recall  f1-score   support

BETWEEN O AND S       0.00      0.00      0.00         3
BETWEEN T AND Z       0.57      0.80      0.67         5

       accuracy                           0.50         8
      macro avg       0.29      0.40      0.33         8
   weighted avg       0.36      0.50      0.42         8



In [295]:
pred_y_2_b

array(['BETWEEN T AND Z', 'BETWEEN T AND Z', 'BETWEEN T AND Z',
       'BETWEEN O AND S', 'BETWEEN T AND Z', 'BETWEEN T AND Z',
       'BETWEEN T AND Z', 'BETWEEN T AND Z'], dtype='<U15')

In [296]:
#Predicting TIER_3_ACTUAL_LETTER given TIER_1_WHICH_HALF = FIRST 14 LETTERS ; TIER_2_LETTER_RANGE = BETWEEN A AND G

In [297]:
# taking train data having TIER_2_LETTER_RANGE = BETWEEN A AND G
train_x_3_a = df_train[df_train['TIER_2_LETTER_RANGE'][train_x_1.index] == 'BETWEEN A AND G'].drop(columns=['text','TIER_1_WHICH_HALF','TIER_2_LETTER_RANGE','TIER_3_ACTUAL_LETTER'])
# taking test data where TIER_1_WHICH_HALF was predicted as FIRST 14 LETTERS
test_x_3_a = df_test[df_test['TIER_2_LETTER_RANGE'][test_x_1.index] == 'BETWEEN A AND G'].drop(columns=['text','TIER_1_WHICH_HALF','TIER_2_LETTER_RANGE','TIER_3_ACTUAL_LETTER','TIER_1_WHICH_HALF_PRED'])

train_y_3_a = df_train[df_train['TIER_2_LETTER_RANGE'][train_x_1.index] == 'BETWEEN A AND G']['TIER_3_ACTUAL_LETTER']
test_y_3_a = df_test[df_test['TIER_2_LETTER_RANGE'][test_x_1.index] == 'BETWEEN A AND G']['TIER_3_ACTUAL_LETTER']


In [302]:
# model_3_AtoG = SVC()
model_3_AtoG = MultinomialNB()
# model_3_AtoG = GaussianNB()
# model_3_AtoG = DecisionTreeClassifier()
# model_3_AtoG = LogisticRegression()
model_3_AtoG.fit(train_x_3_a,train_y_3_a)
pred_y_3_a = model_3_AtoG.predict(test_x_3_a)
print(classification_report(test_y_3_a, pred_y_3_a))

              precision    recall  f1-score   support

           a       0.00      0.00      0.00         1
           b       1.00      1.00      1.00         1
           c       0.00      0.00      0.00         1
           d       0.00      0.00      0.00         0
           e       0.00      0.00      0.00         1
           f       0.00      0.00      0.00         1

    accuracy                           0.20         5
   macro avg       0.17      0.17      0.17         5
weighted avg       0.20      0.20      0.20         5



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [303]:
test_y_3_a

59    a
60    b
61    c
63    e
64    f
Name: TIER_3_ACTUAL_LETTER, dtype: object

In [304]:
pred_y_3_a

array(['d', 'b', 'd', 'd', 'd'], dtype='<U1')

In [305]:
#Predicting TIER_3_ACTUAL_LETTER given TIER_1_WHICH_HALF = FIRST 14 LETTERS ; TIER_2_LETTER_RANGE = BETWEEN H AND N

In [306]:
# taking train data having TIER_2_LETTER_RANGE = BETWEEN A AND G
train_x_3_b = df_train[df_train['TIER_2_LETTER_RANGE'][train_x_1.index] == 'BETWEEN H AND N'].drop(columns=['text','TIER_1_WHICH_HALF','TIER_2_LETTER_RANGE','TIER_3_ACTUAL_LETTER'])
# taking test data where TIER_1_WHICH_HALF was predicted as FIRST 14 LETTERS
test_x_3_b = df_test[df_test['TIER_2_LETTER_RANGE'][test_x_1.index] == 'BETWEEN H AND N'].drop(columns=['text','TIER_1_WHICH_HALF','TIER_2_LETTER_RANGE','TIER_3_ACTUAL_LETTER','TIER_1_WHICH_HALF_PRED'])

train_y_3_b = df_train[df_train['TIER_2_LETTER_RANGE'][train_x_1.index] == 'BETWEEN H AND N']['TIER_3_ACTUAL_LETTER']
test_y_3_b = df_test[df_test['TIER_2_LETTER_RANGE'][test_x_1.index] == 'BETWEEN H AND N']['TIER_3_ACTUAL_LETTER']


In [312]:
# model_3_HtoN = SVC()
# model_3_HtoN = MultinomialNB()
# model_3_HtoN = GaussianNB()
model_3_HtoN = DecisionTreeClassifier()
# model_3_HtoN = LogisticRegression()
model_3_HtoN.fit(train_x_3_b,train_y_3_b)
pred_y_3_b = model_3_HtoN.predict(test_x_3_b)
print(classification_report(test_y_3_b, pred_y_3_b))

              precision    recall  f1-score   support

           i       1.00      1.00      1.00         1
           j       0.00      0.00      0.00         0
           l       0.00      0.00      0.00         1
           m       0.00      0.00      0.00         1

    accuracy                           0.33         3
   macro avg       0.25      0.25      0.25         3
weighted avg       0.33      0.33      0.33         3



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [ ]:
#Predicting TIER_3_ACTUAL_LETTER given TIER_1_WHICH_HALF = LAST 12 LETTERS ; TIER_2_LETTER_RANGE = BETWEEN O AND S

In [313]:
# taking train data having TIER_2_LETTER_RANGE = BETWEEN A AND G
train_x_3_c = df_train[df_train['TIER_2_LETTER_RANGE'][train_x_1.index] == 'BETWEEN O AND S'].drop(columns=['text','TIER_1_WHICH_HALF','TIER_2_LETTER_RANGE','TIER_3_ACTUAL_LETTER'])
# taking test data where TIER_1_WHICH_HALF was predicted as FIRST 14 LETTERS
test_x_3_c = df_test[df_test['TIER_2_LETTER_RANGE'][test_x_1.index] == 'BETWEEN O AND S'].drop(columns=['text','TIER_1_WHICH_HALF','TIER_2_LETTER_RANGE','TIER_3_ACTUAL_LETTER','TIER_1_WHICH_HALF_PRED'])

train_y_3_c = df_train[df_train['TIER_2_LETTER_RANGE'][train_x_1.index] == 'BETWEEN O AND S']['TIER_3_ACTUAL_LETTER']
test_y_3_c = df_test[df_test['TIER_2_LETTER_RANGE'][test_x_1.index] == 'BETWEEN O AND S']['TIER_3_ACTUAL_LETTER']


In [315]:
# model_3_OtoS = SVC()
model_3_OtoS = MultinomialNB()
# model_3_OtoS = GaussianNB()
# model_3_OtoS = DecisionTreeClassifier()
# model_3_OtoS = LogisticRegression()
model_3_OtoS.fit(train_x_3_c,train_y_3_c)
pred_y_3_c = model_3_OtoS.predict(test_x_3_c)
print(classification_report(test_y_3_c, pred_y_3_c))

              precision    recall  f1-score   support

           o       0.50      1.00      0.67         1
           r       1.00      1.00      1.00         1
           s       0.00      0.00      0.00         1

    accuracy                           0.67         3
   macro avg       0.50      0.67      0.56         3
weighted avg       0.50      0.67      0.56         3



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#Predicting TIER_3_ACTUAL_LETTER given TIER_1_WHICH_HALF = LAST 12 LETTERS ; TIER_2_LETTER_RANGE = BETWEEN T AND Z

In [316]:
# taking train data having TIER_2_LETTER_RANGE = BETWEEN T AND Z
train_x_3_d = df_train[df_train['TIER_2_LETTER_RANGE'][train_x_1.index] == 'BETWEEN T AND Z'].drop(columns=['text','TIER_1_WHICH_HALF','TIER_2_LETTER_RANGE','TIER_3_ACTUAL_LETTER'])
# taking test data where TIER_1_WHICH_HALF was predicted as LAST 12 LETTERS
test_x_3_d = df_test[df_test['TIER_2_LETTER_RANGE'][test_x_1.index] == 'BETWEEN T AND Z'].drop(columns=['text','TIER_1_WHICH_HALF','TIER_2_LETTER_RANGE','TIER_3_ACTUAL_LETTER','TIER_1_WHICH_HALF_PRED'])

train_y_3_d = df_train[df_train['TIER_2_LETTER_RANGE'][train_x_1.index] == 'BETWEEN T AND Z']['TIER_3_ACTUAL_LETTER']
test_y_3_d = df_test[df_test['TIER_2_LETTER_RANGE'][test_x_1.index] == 'BETWEEN T AND Z']['TIER_3_ACTUAL_LETTER']


In [317]:
# model_3_TtoZ = SVC()
model_3_TtoZ = MultinomialNB()
# model_3_TtoZ = GaussianNB()
# model_3_TtoZ = DecisionTreeClassifier()
# model_3_TtoZ = LogisticRegression()
model_3_TtoZ.fit(train_x_3_d,train_y_3_d)
pred_y_3_d = model_3_TtoZ.predict(test_x_3_d)
print(classification_report(test_y_3_d, pred_y_3_d))

              precision    recall  f1-score   support

           v       1.00      0.50      0.67         2
           w       0.00      0.00      0.00         0
           x       0.00      0.00      0.00         1
           z       0.67      1.00      0.80         2

    accuracy                           0.60         5
   macro avg       0.42      0.38      0.37         5
weighted avg       0.67      0.60      0.59         5



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [318]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from collections import Counter
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

# Basic Preprocessing function
def preprocess(text):
    text = re.sub(r'[^\w\s]', '', text).lower()
    tokens = nltk.word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stopwords.words('english')]
    return ' '.join(lemmatized_tokens)

data = pd.read_csv('train_data.csv')

data['clean_text'] = data['text'].apply(preprocess)

# Vectorize sentences
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data['clean_text'])

# Encode labels
le_tier1 = LabelEncoder()
le_tier2 = LabelEncoder()
le_tier3 = LabelEncoder()

y_tier1 = le_tier1.fit_transform(data['TIER_1_WHICH_HALF'])
y_tier2 = le_tier2.fit_transform(data['TIER_2_LETTER_RANGE'])
y_tier3 = le_tier3.fit_transform(data['TIER_3_ACTUAL_LETTER'])

# Train the decision tree classifiers
clf_tier1 = DecisionTreeClassifier()
clf_tier1.fit(X, y_tier1)

clf_tier2_first14 = DecisionTreeClassifier()
clf_tier2_last12 = DecisionTreeClassifier()

clf_tier2_first14.fit(X[data['TIER_1_WHICH_HALF'] == 'FIRST 14 LETTERS'], y_tier2[data['TIER_1_WHICH_HALF'] == 'FIRST 14 LETTERS'])
clf_tier2_last12.fit(X[data['TIER_1_WHICH_HALF'] == 'LAST 12 LETTERS'], y_tier2[data['TIER_1_WHICH_HALF'] == 'LAST 12 LETTERS'])

# Create clfs_tier3 dictionary
clfs_tier3 = {}
for key in le_tier2.classes_:
    idx = (data['TIER_2_LETTER_RANGE'] == key)
    clf = DecisionTreeClassifier()
    clf.fit(X[idx], y_tier3[idx])
    clfs_tier3[key] = clf

# Prediction function
def predict_encoded_letter(sentence):
    clean_sentence = preprocess(sentence)
    x = vectorizer.transform([clean_sentence])

    tier1_pred = clf_tier1.predict(x)
    tier1_pred_label = le_tier1.inverse_transform(tier1_pred)[0]

    if tier1_pred_label == 'FIRST 14 LETTERS':
        tier2_pred = clf_tier2_first14.predict(x)
    else:
        tier2_pred = clf_tier2_last12.predict(x)

    tier2_pred_label = le_tier2.inverse_transform(tier2_pred)[0]

    tier3_pred = clfs_tier3[tier2_pred_label].predict(x)
    tier3_pred_label = le_tier3.inverse_transform(tier3_pred)[0]

    return tier3_pred_label

# Get top 10 keywords/entities for each bucket
def get_top_keywords(data, labels):
    counters = {label: Counter() for label in labels}

    for label, counter in counters.items():
        idx = (data['TIER_2_LETTER_RANGE'] == label)
        sentences = data['clean_text'][idx]

        for sentence in sentences:
            tokens = sentence.split()
            for token in tokens:
                counter[token] += 1

    top_keywords = {label: counter.most_common(10) for label, counter in counters.items()}
    return top_keywords

# Example usage
test_sentence = "What is the capital of France?"
predicted_encoded_letter = predict_encoded_letter(test_sentence)
print(f"Predicted encoded letter: {predicted_encoded_letter}")

tier2_labels = data['TIER_2_LETTER_RANGE'].unique()
top_keywords = get_top_keywords(data, tier2_labels)
print("Top 10 keywords for each bucket:")
for label, keywords in top_keywords.items():
    print(f"{label}: {keywords}")


Predicted encoded letter: d
Top 10 keywords for each bucket:
BETWEEN T AND Z: [('u', 3), ('capital', 2), ('named', 2), ('stand', 2), ('country', 1), ('mikhail', 1), ('gorbachev', 1), ('leader', 1), ('city', 1), ('two', 1)]
BETWEEN A AND G: [('grow', 2), ('tip', 2), ('company', 1), ('us', 1), ('unfamiliar', 1), ('face', 1), ('celebrity', 1), ('spokespeople', 1), ('promote', 1), ('use', 1)]
BETWEEN H AND N: [('name', 1), ('four', 1), ('horseman', 1), ('apocalypse', 1), ('much', 1), ('canadian', 1), ('1967', 1), ('twenty', 1), ('dollar', 1), ('gold', 1)]
BETWEEN O AND S: [('nafta', 1), ('stand', 1), ('caliente', 1), ('mean', 1), ('english', 1), ('softest', 1), ('part', 1), ('body', 1), ('wrote', 1), ('scarlet', 1)]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [319]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report

# Load and preprocess the test data
test_data = pd.read_csv("test_data.csv")
test_data['clean_text'] = test_data['text'].apply(preprocess)

# Extract features from the test data
X_test = vectorizer.transform(test_data['clean_text'])

# Make predictions for the test data
y_test_true = test_data['TIER_3_ACTUAL_LETTER']
y_test_pred = [predict_encoded_letter(sentence) for sentence in test_data['text']]

# Calculate evaluation metrics
accuracy = accuracy_score(y_test_true, y_test_pred)
print(f"Accuracy: {accuracy:.2f}")

report = classification_report(y_test_true, y_test_pred)
print("Classification report:")
print(report)


Accuracy: 0.06
Classification report:
              precision    recall  f1-score   support

           a       1.00      1.00      1.00         1
           b       0.00      0.00      0.00         1
           c       0.00      0.00      0.00         1
           d       0.00      0.00      0.00         0
           e       0.00      0.00      0.00         1
           f       0.00      0.00      0.00         1
           g       0.00      0.00      0.00         0
           i       0.00      0.00      0.00         1
           j       0.00      0.00      0.00         0
           l       0.00      0.00      0.00         1
           m       0.00      0.00      0.00         1
           o       0.00      0.00      0.00         1
           r       0.00      0.00      0.00         1
           s       0.00      0.00      0.00         1
           v       0.00      0.00      0.00         2
           x       0.00      0.00      0.00         1
           z       0.00      0.00      0.00

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.